In [96]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [98]:
!pip install konlpy

In [99]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt

In [100]:
df_2019 = pd.read_csv('/content/drive/MyDrive/핀인사이트 데이터분석/2019년 기사(완료).csv')
df_2019.drop(columns=['Unnamed: 0'],inplace=True)
df_2019.reset_index(drop=True, inplace=True)
df_2019

,date,url,category,title,content,tokenize_title,tokenize_content
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,네이버·카카오 민간 블록체인 급팽창…디앱상용화 '잰걸음',2019 블록체인 시장을 진단한다 '사실 앞에 겸손한 정통 민영 뉴스통신' 이 ...,"['네이버', '카카오', '민간', '블록', '체인', '급', '팽창', '디...","['블록', '체인', '시장', '진단', '한다', '사실', '앞', '겸손한..."
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,[국고처 한동훈] 종목상담 - 카카오(035720),종목상담 카카오(035720) 12월 28일 방송된 '국민주식고충처리반'에 출연...,"['국고', '처', '한동훈', '종목', '상담', '카카오']","['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충..."
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,"카카오, 카카오T `카풀혁명` 원년으로",2019 재계가 뛴다 카카오는 블록체인 기술 계열사 '그라운드X'를 설립하고 ...,"['카카오', '카카오', '카풀', '혁명', '원년']","['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라..."
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,[CEO] 카카오의 세계화… 지구촌 휩쓰는 '캐릭터 장인',카카오IX가 '카카오프렌즈'를 앞세운 캐릭터사업으로 본격적인 해외 공략에 나섰다. ...,"['카카오', '세계', '화', '지구촌', '휩쓰는', '캐릭터', '장인']","['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'..."
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,"김성수 전 CJ ENM 대표, 카카오 콘텐츠 사령탑 맡는다",배우 매니지먼트 3개사‥광고모델 캐스팅 에이전시 인수 완료 김성수 카카오 전 CJ...,"['김성수', '대표', '카카오', '콘텐츠', '사령탑', '맡는다']","['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시..."
...,...,...,...,...,...,...,...
3714,2019-12-30T17:27:00,https://n.news.naver.com/mnews/article/469/000...,사회,"노조반발 대구 카카오택시, 새해 이틀 앞두고 극적 타결","노조 규탄집회 잇단 취소 끝 ""복지 증진 대시민 서비스 향상 노력"" 합의 가맹형 플...","['노조', '반발', '대구', '카카오', '택시', '새해', '이틀', '앞...","['노조', '규탄', '집회', '잇단', '취소', '끝', '복지', '증진'..."
3715,2019-12-31T06:46:00,https://n.news.naver.com/mnews/article/417/000...,IT/과학,"The New 카카오, 도전은 계속된다",올 들어 유명인들이 스스로 목숨을 끊는 사건이 연달아 발생하면서 악성댓글에 대한 위...,"['카카오', '도전', '계속']","['올', '들어', '유명인', '스스로', '목숨', '끊는', '사건', '연..."
3716,2019-12-31T09:59:00,https://n.news.naver.com/mnews/article/277/000...,IT/과학,"이동걸 산은 회장, 카카오모빌리티 비공식 방문",판교 사옥서 관계자들과 간담 금융권 우호적 신호 주목 이동걸 산업은행 회장이 최근...,"['이동걸', '산', '회장', '카카오', '모빌리티', '비공식', '방문']","['판교', '사옥', '서', '관계자', '간담', '금융', '권', '우호'..."
3717,2019-12-31T10:28:00,https://n.news.naver.com/mnews/article/421/000...,IT/과학,숨은 작가 찾았다…카카오 브런치 출판 수상작 공개,카카오의 글쓰기 플랫폼 브런치가 '제7회 브런치북 출판 프로젝트' 수상작 15편을 ...,"['숨은', '작가', '카카오', '브런치', '출판', '수상작', '공개']","['카카오', '글쓰기', '플랫폼', '브런치', '제', '브런치', '북', ..."


In [101]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [102]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df_2019['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['네이버', '카카오', '민간', '블록', '체인', '급', '팽창', '디앱', '상용', '화', '잰', '걸음'],
 ['국고', '처', '한동훈', '종목', '상담', '카카오'],
 ['카카오', '카카오', '카풀', '혁명', '원년'],
 ['카카오', '세계', '화', '지구촌', '휩쓰는', '캐릭터', '장인'],
 ['김성수', '대표', '카카오', '콘텐츠', '사령탑', '맡는다'],
 ['비즈', '김성수', '대표', '카카오', '신임', '대표', '선임'],
 ['김성수', '카카오', '신임', '대표'],
 ['공식', '카카오', '김성수', '신임', '대표', '취임', '글로벌', '콘텐츠', '강', '성장'],
 ['카카오', '카카오', '해외', '피싱', '방지', '글로브', '시그널', '기능', '도입'],
 ['단독',
  '카카오',
  '이병헌',
  '이어',
  '송중기',
  '박보검',
  '블러',
  '썸',
  '인수',
  '임박',
  '엔터',
  '지각',
  '변동'],
 ['프로필', '지구본', '뜨면', '사기', '조심', '카카오', '글로브', '시그널', '도입'],
 ['신년', '기획', '카카오', '페이', '대표', '불편', '걷어내니', '실적', '제발'],
 ['시그널', '카카오', '엠', '들여', '인기', '연예인', '소속사', '대거', '인수'],
 ['이슈', '분석', '카카오', '뱅크', '혈관', '전산', '센터', '가다'],
 ['공', '회전', '카카오', '카풀', '새해', '캄캄한', '앞길'],
 ['카카오', '게임', '대표', '세', '계시', '서', '독보', '적', '캐', '주얼', '게임', '되겠다'],
 ['김성수', '카카오', '신임', '대표', '선임', '제작', '역량', '글로벌', '키운다'],
 ['신년사', '카카오', '게임', '나이키', '유저',

In [105]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<4669 unique tokens: ['걸음', '급', '네이버', '디앱', '민간']...>


In [106]:
for value in id2word_title:
  print(value, id2word_title[value])

0 걸음
1 급
2 네이버
3 디앱
4 민간
5 블록
6 상용
7 잰
8 체인
9 카카오
10 팽창
11 화
12 국고
13 상담
14 종목
15 처
16 한동훈
17 원년
18 카풀
19 혁명
20 세계
21 장인
22 지구촌
23 캐릭터
24 휩쓰는
25 김성수
26 대표
27 맡는다
28 사령탑
29 콘텐츠
30 비즈
31 선임
32 신임
33 강
34 공식
35 글로벌
36 성장
37 취임
38 글로브
39 기능
40 도입
41 방지
42 시그널
43 피싱
44 해외
45 단독
46 박보검
47 변동
48 블러
49 송중기
50 썸
51 엔터
52 이병헌
53 이어
54 인수
55 임박
56 지각
57 뜨면
58 사기
59 조심
60 지구본
61 프로필
62 걷어내니
63 기획
64 불편
65 신년
66 실적
67 제발
68 페이
69 대거
70 들여
71 소속사
72 엠
73 연예인
74 인기
75 가다
76 뱅크
77 분석
78 센터
79 이슈
80 전산
81 혈관
82 공
83 새해
84 앞길
85 캄캄한
86 회전
87 게임
88 계시
89 독보
90 되겠다
91 서
92 세
93 적
94 주얼
95 캐
96 역량
97 제작
98 키운다
99 나이키
100 도전
101 신년사
102 유저
103 층
104 해
105 향
106 쥬얼
107 집중
108 하이퍼
109 한다
110 기틀
111 대중
112 마련
113 전문
114 한발
115 구형
116 오픈넷
117 유감
118 유죄
119 이석우
120 자리매
121 대표이사
122 새
123 씨
124 법
125 보호
126 성
127 아동
128 위반
129 청소년
130 거듭
131 나
132 동영상
133 제작사
134 되
135 만나
136 미
137 살아났다
138 시절
139 완결
140 작품
141 카카오페이지
142 통신
143 나선
144 종합
145 넘어
146 놀이
147 만든다
148 밀착
149 삶
150 생활
151 앱
152 요소
153 활용
154 노림수
155 못

In [107]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1)],
 [(9, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)],
 [(9, 2), (17, 1), (18, 1), (19, 1)],
 [(9, 1), (11, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)],
 [(9, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)],
 [(9, 1), (25, 1), (26, 2), (30, 1), (31, 1), (32, 1)],
 [(9, 1), (25, 1), (26, 1), (32, 1)],
 [(9, 1),
  (25, 1),
  (26, 1),
  (29, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1)],
 [(9, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)],
 [(9, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1)],
 [(9, 1),
  (38, 1),
  (40, 1),
  (42, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(9, 1),
  (26, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1)],
 [(9, 1),
  (42, 1),
  (54, 1),
  (69, 1),


In [108]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.012*"통신" + 0.009*"상담" + 0.007*"교통" + 0.007*"챗봇" + 0.007*"자본" + 0.007*"하세요" + 0.006*"납부" + 0.006*"전쟁" + 0.006*"페이" + 0.005*"호조"
1 : 0.011*"기아차" + 0.010*"주주" + 0.010*"뱅크" + 0.009*"승인" + 0.009*"모빌리티" + 0.008*"클래식" + 0.008*"대형" + 0.008*"게임" + 0.007*"테라" + 0.007*"조작"
2 : 0.013*"협력" + 0.010*"외국인" + 0.009*"대표" + 0.008*"거래소" + 0.007*"네이버" + 0.007*"증인" + 0.007*"매수" + 0.006*"인터넷" + 0.006*"서도" + 0.005*"스타"
3 : 0.009*"카뱅" + 0.008*"주주" + 0.007*"뱅크" + 0.006*"최대" + 0.006*"게임" + 0.006*"문화" + 0.006*"거래" + 0.005*"제휴" + 0.005*"지원" + 0.005*"손잡고"
4 : 0.013*"의장" + 0.012*"김범수" + 0.010*"협력" + 0.010*"체결" + 0.009*"열사" + 0.008*"신고" + 0.008*"무죄" + 0.006*"누락" + 0.006*"기술" + 0.006*"교환"
5 : 0.013*"대한항공" + 0.012*"대구" + 0.009*"손잡고" + 0.008*"뱅크" + 0.008*"택시" + 0.007*"타다" + 0.006*"주주" + 0.006*"자본" + 0.006*"최대" + 0.005*"산업"
6 : 0.016*"최대" + 0.015*"주주" + 0.015*"뱅크" + 0.008*"변경" + 0.007*"김범수" + 0.006*"택시" + 0.006*"의장" + 0.005*"이자" + 0.005*"적금" + 0.005*"반대"
7 : 0.008*"교환" + 0.008*"출시" + 0.007*"투" + 0.006*"카뱅" + 0.006*

In [109]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.023976639),
 (1, 0.023987569),
 (2, 0.02398043),
 (3, 0.024003096),
 (4, 0.02397682),
 (5, 0.7841373),
 (6, 0.023976017),
 (7, 0.023982983),
 (8, 0.023995994),
 (9, 0.023983166)]

In [110]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

[(0, 0.023976665), (1, 0.023988687), (2, 0.023980461), (3, 0.02400831), (4, 0.023976846), (5, 0.78413206), (6, 0.023976043), (7, 0.023983046), (8, 0.023994701), (9, 0.023983197)]
[(0, 0.71813565), (1, 0.031317867), (2, 0.031317815), (3, 0.031317845), (4, 0.031317815), (5, 0.03131889), (6, 0.031319637), (7, 0.031317815), (8, 0.031317826), (9, 0.031318855)]
[(0, 0.03772868), (1, 0.037722576), (2, 0.03772358), (3, 0.0377231), (4, 0.037723087), (5, 0.037723944), (6, 0.03772245), (7, 0.037722424), (8, 0.6604867), (9, 0.037723493)]
[(0, 0.029556619), (1, 0.029556127), (2, 0.02955608), (3, 0.029556394), (4, 0.7339894), (5, 0.029556906), (6, 0.029556455), (7, 0.029556276), (8, 0.029556785), (9, 0.029558955)]
[(0, 0.032017544), (1, 0.032019008), (2, 0.032015957), (3, 0.03201759), (4, 0.03201562), (5, 0.032015838), (6, 0.03201775), (7, 0.032016657), (8, 0.03201838), (9, 0.7118457)]
[(0, 0.03108651), (1, 0.031088546), (2, 0.031090489), (3, 0.03109166), (4, 0.031088594), (5, 0.031087104), (6, 0.03

In [ ]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df_2019.loc[i, 'topic(title)'] = dominant_topic[0]
  df_2019.loc[i, 'topic_details(title)'] = dominant_topic[1]
df_2019

,date,url,title,content,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,네이버·카카오 민간 블록체인 급팽창…디앱상용화 '잰걸음',2019 블록체인 시장을 진단한다 '사실 앞에 겸손한 정통 민영 뉴스통신' 이 ...,"['네이버', '카카오', '민간', '블록', '체인', '급', '팽창', '디...","['블록', '체인', '시장', '진단', '한다', '사실', '앞', '겸손한...",1.0,0.783690
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,[국고처 한동훈] 종목상담 - 카카오(035720),종목상담 카카오(035720) 12월 28일 방송된 '국민주식고충처리반'에 출연...,"['국고', '처', '한동훈', '종목', '상담', '카카오']","['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충...",0.0,0.718134
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"카카오, 카카오T `카풀혁명` 원년으로",2019 재계가 뛴다 카카오는 블록체인 기술 계열사 '그라운드X'를 설립하고 ...,"['카카오', '카카오', '카풀', '혁명', '원년']","['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라...",9.0,0.660647
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,[CEO] 카카오의 세계화… 지구촌 휩쓰는 '캐릭터 장인',카카오IX가 '카카오프렌즈'를 앞세운 캐릭터사업으로 본격적인 해외 공략에 나섰다. ...,"['카카오', '세계', '화', '지구촌', '휩쓰는', '캐릭터', '장인']","['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'...",9.0,0.733957
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"김성수 전 CJ ENM 대표, 카카오 콘텐츠 사령탑 맡는다",배우 매니지먼트 3개사‥광고모델 캐스팅 에이전시 인수 완료 김성수 카카오 전 CJ...,"['김성수', '대표', '카카오', '콘텐츠', '사령탑', '맡는다']","['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시...",6.0,0.711895
...,...,...,...,...,...,...,...,...
3714,2019-12-30T17:27:00,https://n.news.naver.com/mnews/article/469/000...,"노조반발 대구 카카오택시, 새해 이틀 앞두고 극적 타결","노조 규탄집회 잇단 취소 끝 ""복지 증진 대시민 서비스 향상 노력"" 합의 가맹형 플...","['노조', '반발', '대구', '카카오', '택시', '새해', '이틀', '앞...","['노조', '규탄', '집회', '잇단', '취소', '끝', '복지', '증진'...",2.0,0.428548
3715,2019-12-31T06:46:00,https://n.news.naver.com/mnews/article/417/000...,"The New 카카오, 도전은 계속된다",올 들어 유명인들이 스스로 목숨을 끊는 사건이 연달아 발생하면서 악성댓글에 대한 위...,"['카카오', '도전', '계속']","['올', '들어', '유명인', '스스로', '목숨', '끊는', '사건', '연...",4.0,0.627074
3716,2019-12-31T09:59:00,https://n.news.naver.com/mnews/article/277/000...,"이동걸 산은 회장, 카카오모빌리티 비공식 방문",판교 사옥서 관계자들과 간담 금융권 우호적 신호 주목 이동걸 산업은행 회장이 최근...,"['이동걸', '산', '회장', '카카오', '모빌리티', '비공식', '방문']","['판교', '사옥', '서', '관계자', '간담', '금융', '권', '우호'...",3.0,0.523369
3717,2019-12-31T10:28:00,https://n.news.naver.com/mnews/article/421/000...,숨은 작가 찾았다…카카오 브런치 출판 수상작 공개,카카오의 글쓰기 플랫폼 브런치가 '제7회 브런치북 출판 프로젝트' 수상작 15편을 ...,"['숨은', '작가', '카카오', '브런치', '출판', '수상작', '공개']","['카카오', '글쓰기', '플랫폼', '브런치', '제', '브런치', '북', ...",7.0,0.519386


In [ ]:
# 토픽별 비중 확인
df_2019['topic(title)'].unique()

array([1., 0., 9., 6., 4., 7., 5., 8., 3., 2.])

In [ ]:
df_2019['topic(title)'].value_counts()

1.0    674
4.0    458
5.0    429
2.0    346
8.0    334
3.0    319
9.0    313
7.0    313
6.0    293
0.0    240
Name: topic(title), dtype: int64

In [ ]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df_2019['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<26541 unique tokens: ['가기', '가는', '가늠', '가능성', '가능하도록']...>


In [ ]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
21541 들으려는
21542 뜬
21543 로딩
21544 류한나
21545 만렙
21546 머리카락
21547 바텀업
21548 배우려는
21549 벌여야
21550 뽑는다면
21551 사고력
21552 성장하지
21553 습득
21554 신충
21555 심심하면
21556 쓸모없어지는
21557 앳된
21558 역할극
21559 원한다
21560 자질
21561 재밌을
21562 전투력
21563 쫑긋
21564 최승
21565 추론
21566 취준생
21567 커밍
21568 학업
21569 해보라
21570 해줘야
21571 둔치
21572 시가지
21573 표고
21574 가을
21575 갈아엎어야
21576 겨울
21577 고칠테니
21578 궁금하게
21579 놓친다
21580 되겠지라고
21581 될거다
21582 맞는지
21583 명료
21584 모아놓고
21585 뱉은
21586 보여줬는데
21587 야하고
21588 어려워서
21589 여길
21590 익숙하게
21591 있듯
21592 잘라서
21593 조인트
21594 좁히는
21595 찾아야하고
21596 채워져
21597 특강
21598 헤어질
21599 회고
21600 김춘수
21601 이성원
21602 과업
21603 디플로
21604 매끄러운
21605 매끄럽고
21606 미니미
21607 상호작용
21608 스루
21609 스몰톡
21610 심슨
21611 좁고
21612 컨버런스
21613 할지까지
21614 흐르듯
21615 흘러감을
21616 드러나는
21617 드러내기
21618 들을까
21619 매끄럽게
21620 태어나면
21621 피곤해
21622 흘러가는
21623 가능할지에
21624 난이도
21625 듀플렉스
21626 볼실
21627 이끌었듯
21628 이해해
21629 월렛용
21630 카스
21631 하드
21632 간결해지고
21633 꽂아
21634 꽂아보다
21635 뉴럴
21636 

In [ ]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.003*"기아차" + 0.001*"종목" + 0.001*"노조" + 0.001*"외국인" + 0.001*"삼성" + 0.001*"브런치" + 0.001*"택시" + 0.001*"상위" + 0.001*"기아" + 0.001*"거래소"
1 : 0.003*"게임" + 0.002*"예약" + 0.002*"테라" + 0.002*"사전" + 0.002*"카카오페이지" + 0.002*"클래식" + 0.001*"택시" + 0.001*"매도" + 0.001*"폐지" + 0.001*"매출"
2 : 0.004*"게임" + 0.004*"택시" + 0.003*"매출" + 0.002*"유플러스" + 0.002*"달빛조각사" + 0.002*"체인" + 0.002*"블록" + 0.002*"기" + 0.002*"사업" + 0.002*"모빌리티"
3 : 0.006*"뱅크" + 0.004*"금융" + 0.003*"지" + 0.003*"주주" + 0.003*"은행" + 0.003*"한국" + 0.002*"보유" + 0.002*"택시" + 0.002*"투자" + 0.002*"인터넷"
4 : 0.005*"텔레콤" + 0.003*"매출" + 0.002*"대비" + 0.002*"뱅크" + 0.002*"기" + 0.002*"증가" + 0.002*"이익" + 0.002*"영업" + 0.002*"동기" + 0.002*"거래"
5 : 0.002*"보험" + 0.002*"페이" + 0.002*"체인" + 0.002*"블록" + 0.001*"화재" + 0.001*"타이어" + 0.001*"클레이튼" + 0.001*"서비스" + 0.001*"금융" + 0.001*"삼"
6 : 0.003*"의장" + 0.003*"김" + 0.003*"클레이튼" + 0.002*"허위" + 0.002*"제출" + 0.002*"재판" + 0.002*"선고" + 0.002*"체인" + 0.002*"무죄" + 0.002*"자료"
7 : 0.004*"납부" + 0.003*"뱅크" + 0.003*"고객" + 0.003*"페이" + 0

In [ ]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(0, 0.0240206),
 (1, 0.78369385),
 (2, 0.024020156),
 (3, 0.024065698),
 (4, 0.024025891),
 (5, 0.024021834),
 (6, 0.024021264),
 (7, 0.024023796),
 (8, 0.024085818),
 (9, 0.024021039)]

In [ ]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

[(0, 0.024020573), (1, 0.7836984), (2, 0.02402013), (3, 0.024062766), (4, 0.024025865), (5, 0.02402181), (6, 0.02402124), (7, 0.02402377), (8, 0.024084516), (9, 0.024021015)]
[(0, 0.71813565), (1, 0.03131766), (2, 0.03131824), (3, 0.03131828), (4, 0.03131764), (5, 0.031318184), (6, 0.03131775), (7, 0.031317607), (8, 0.031317633), (9, 0.031321395)]
[(0, 0.03770575), (1, 0.037705928), (2, 0.03770662), (3, 0.03770803), (4, 0.03770621), (5, 0.03770707), (6, 0.037704658), (7, 0.03770438), (8, 0.037704073), (9, 0.6606473)]
[(0, 0.029555177), (1, 0.029562965), (2, 0.029555395), (3, 0.029560568), (4, 0.029559445), (5, 0.029574417), (6, 0.029556807), (7, 0.029558951), (8, 0.029558325), (9, 0.73395795)]
[(0, 0.032010008), (1, 0.03201074), (2, 0.0320108), (3, 0.032011244), (4, 0.03201593), (5, 0.032010395), (6, 0.71189445), (7, 0.032012437), (8, 0.032012377), (9, 0.032011617)]
[(0, 0.031093692), (1, 0.21411881), (2, 0.031101875), (3, 0.031096162), (4, 0.5371042), (5, 0.031094525), (6, 0.03110284)

In [ ]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df_2019.loc[i, 'topic(content)'] = dominant_topic[0]
  df_2019.loc[i, 'topic_details(content)'] = dominant_topic[1]

df_2019

,date,url,title,content,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2019-01-01T07:31:00,https://news.naver.com/main/read.nhn?mode=LS2D...,네이버·카카오 민간 블록체인 급팽창…디앱상용화 '잰걸음',2019 블록체인 시장을 진단한다 '사실 앞에 겸손한 정통 민영 뉴스통신' 이 ...,"['네이버', '카카오', '민간', '블록', '체인', '급', '팽창', '디...","['블록', '체인', '시장', '진단', '한다', '사실', '앞', '겸손한...",1.0,0.783690,1.0,0.783693
1,2019-01-01T13:15:00,https://news.naver.com/main/read.nhn?mode=LS2D...,[국고처 한동훈] 종목상담 - 카카오(035720),종목상담 카카오(035720) 12월 28일 방송된 '국민주식고충처리반'에 출연...,"['국고', '처', '한동훈', '종목', '상담', '카카오']","['종목', '상담', '카카오', '방송', '된', '국민', '주식', '고충...",0.0,0.718134,0.0,0.718136
2,2019-01-02T04:03:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"카카오, 카카오T `카풀혁명` 원년으로",2019 재계가 뛴다 카카오는 블록체인 기술 계열사 '그라운드X'를 설립하고 ...,"['카카오', '카카오', '카풀', '혁명', '원년']","['재계', '뛴다', '카카오', '블록', '체인', '기술', '열사', '라...",9.0,0.660647,9.0,0.660647
3,2019-01-02T06:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,[CEO] 카카오의 세계화… 지구촌 휩쓰는 '캐릭터 장인',카카오IX가 '카카오프렌즈'를 앞세운 캐릭터사업으로 본격적인 해외 공략에 나섰다. ...,"['카카오', '세계', '화', '지구촌', '휩쓰는', '캐릭터', '장인']","['카카오', '카카오', '프렌즈', '앞세운', '캐릭터', '사업', '본격'...",9.0,0.733957,9.0,0.733958
4,2019-01-02T09:41:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"김성수 전 CJ ENM 대표, 카카오 콘텐츠 사령탑 맡는다",배우 매니지먼트 3개사‥광고모델 캐스팅 에이전시 인수 완료 김성수 카카오 전 CJ...,"['김성수', '대표', '카카오', '콘텐츠', '사령탑', '맡는다']","['배우', '매니지먼트', '개사', '광고', '모델', '캐스팅', '에이전시...",6.0,0.711895,6.0,0.711895
...,...,...,...,...,...,...,...,...,...,...
3714,2019-12-30T17:27:00,https://n.news.naver.com/mnews/article/469/000...,"노조반발 대구 카카오택시, 새해 이틀 앞두고 극적 타결","노조 규탄집회 잇단 취소 끝 ""복지 증진 대시민 서비스 향상 노력"" 합의 가맹형 플...","['노조', '반발', '대구', '카카오', '택시', '새해', '이틀', '앞...","['노조', '규탄', '집회', '잇단', '취소', '끝', '복지', '증진'...",2.0,0.428548,2.0,0.428492
3715,2019-12-31T06:46:00,https://n.news.naver.com/mnews/article/417/000...,"The New 카카오, 도전은 계속된다",올 들어 유명인들이 스스로 목숨을 끊는 사건이 연달아 발생하면서 악성댓글에 대한 위...,"['카카오', '도전', '계속']","['올', '들어', '유명인', '스스로', '목숨', '끊는', '사건', '연...",4.0,0.627074,4.0,0.627076
3716,2019-12-31T09:59:00,https://n.news.naver.com/mnews/article/277/000...,"이동걸 산은 회장, 카카오모빌리티 비공식 방문",판교 사옥서 관계자들과 간담 금융권 우호적 신호 주목 이동걸 산업은행 회장이 최근...,"['이동걸', '산', '회장', '카카오', '모빌리티', '비공식', '방문']","['판교', '사옥', '서', '관계자', '간담', '금융', '권', '우호'...",3.0,0.523369,3.0,0.523299
3717,2019-12-31T10:28:00,https://n.news.naver.com/mnews/article/421/000...,숨은 작가 찾았다…카카오 브런치 출판 수상작 공개,카카오의 글쓰기 플랫폼 브런치가 '제7회 브런치북 출판 프로젝트' 수상작 15편을 ...,"['숨은', '작가', '카카오', '브런치', '출판', '수상작', '공개']","['카카오', '글쓰기', '플랫폼', '브런치', '제', '브런치', '북', ...",7.0,0.519386,7.0,0.520460


In [ ]:
# 토픽별 비중 확인
df_2019['topic(content)'].unique()

array([1., 0., 9., 6., 4., 7., 5., 8., 3., 2.])

In [ ]:
df_2019['topic(content)'].value_counts()

1.0    680
4.0    451
5.0    428
2.0    345
8.0    334
3.0    321
7.0    315
9.0    310
6.0    293
0.0    242
Name: topic(content), dtype: int64